In [1]:
from keras.layers import Dense, Activation, Embedding
from keras.preprocessing import sequence
from collections import defaultdict
from keras.models import Sequential
from keras.layers import LSTM,GRU,Bidirectional
from gensim import corpora
import numpy as np
import pickle,re

Using TensorFlow backend.
d:\installations\miniconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
stoplist = set('a is i of the am'.split())
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower())
    tokenized = [w for w in text.split() if w not in stoplist]
    return tokenized


In [ ]:
docs, y = [], []
with open('./movie_data.txt','r',encoding='latin-1') as csv:
	next(csv)
	for line in csv:
		text,label = line[:-3],int(line[-2])
		text = tokenizer(text)
		docs.append(text)
		y.append(label)